# Test

## `pytest`

Tredjepartsbiblioteket `pytest` kan behöva installeras i vårt *environment*.

In [ ]:
try:
    import pytest
except ImportError:
    ! pip install pytest  # "!" först i en notebook-cell innebär att vi vill köra koden i terminalen

`assert` är ett nyckelord som bland annat används vid tester.

Om uttrycket som följer ett `assert`-*statement* utvärderas till `False` lyfts ett `AssertionError`. Annars händer ingenting.

In [ ]:
assert True

In [ ]:
assert False

In [ ]:
# test.py

def add_one(x):
    return x + 1

def test_add_one():
    assert add_one(4) == 5

In [ ]:
! pytest test.py

Vi kan skapa en `DataCleaner`-klass som har våra funktioner från förra genomgången som metoder.

In [ ]:
# datacleaner.py

import datetime
import logging
import pandas as pd

class DataCleaner:
    def __init__(self, data) -> None:
        self.data = data
        self.logger = logging.getLogger(__name__)

    def clean_dates(self) -> None:
        for row in self.data:
            if '-' in row['date']:
                try:
                    y, m, d = [int(x) for x in row['date'].split('-')]
                except Exception as e:
                    self.logger.error((e, row['date']))
                    new_date = pd.NaT
                else:
                    new_date = datetime.date(y, m, d)
            elif '/' in row['date']:
                try:
                    m, d, y = [int(x) for x in row['date'].split('/')]
                except Exception as e:
                    self.logger.error((e, row['date']))
                    new_date = pd.NaT
                else:
                    new_date = datetime.date(2000 + y, m, d)
            else:
                self.logger.error('Could not parse date %s', row['date'])
                new_date = pd.NaT
            row['date'] = new_date

    def clean_temps(self):
        for row in self.data:
            try:
                new_temp = float(row['temp'])
            except TypeError:  
                new_temp = pd.NA
            except ValueError:  
                new_temp = float(row['temp'].replace(',', '.'))
            except Exception as e:  
                self.logger.error(e)
                new_temp = pd.NA

            row['temp'] = new_temp

    def clean(self):
        self.clean_dates()
        self.clean_temps()
        return self.data


`test_data_cleaner.py` innehåller tester för att se om vår `DataCleaner`-class kan hantera data som är formaterad olika.

In [ ]:
# test_data_cleaner.py

import json
from datacleaner import DataCleaner

data = json.load(open('data/temps2.json'))

def test_data_cleaner_dates():
    dc = DataCleaner(data=data)
    dc.clean_dates()

def test_data_cleaner_temps():
    dc = DataCleaner(data=data)
    dc.clean_temps()

Vi kan köra pytest från en notebook. För att kunna se våra logg-meddelanden behöver vi aktivera *live log* genom att ange `log_cli=true` när vi kör pytest.

In [ ]:
! pytest -o log_cli=true test_data_cleaner.py